## 17 메모리 관리와 순환 참조

### 17.1 메모리 관리
+ 파이썬은 필요없는 객체를 메모리에서 자동으로 삭제한다.           
+ 방식은 두 가지
  + 하나는 참조 수를 세는 방식
  + 하나는 세대를 기준으로 쓸모 없어진 객체를 회수하는 방식


### 17.2 참조 카운트 방식의 메모리 관리

In [1]:
class obj :
  pass

def f(x):
  print(x)

In [2]:
a = obj()
f(a)

In [3]:
a = None

In [4]:
a = obj()
b = obj()
c = obj()
a.b = b
b.c = c
a = b = c = None

### 17.3 순환참조


In [5]:
a = obj()
b = obj()
c = obj()
a.b = b
b.c = c
c.a = a
a=b=c=None

### 17.4 weakref 모듈

In [6]:
import weakref
import numpy as np


In [38]:
a = np.array([1,2,3])
b = weakref.ref(a)
print(b)
print(b())
a = None
print(b)

<weakref at 0x7f2531f0be30; to 'numpy.ndarray' at 0x7f2531f657b0>
[1 2 3]
<weakref at 0x7f2531f0be30; dead>


In [22]:
class Variable:
  def __init__(self, data) :
    if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
    self.data = data
    self.grad = None
    self.creator = None
    #세대 수 표현
    self.generation = 0
  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1
  
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key = lambda x: x.generation)

    add_func(self.creator)
    while funcs:
      f = funcs.pop()
      # output이 많으니까 차례차례 진행
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        # 튜플 변환
        gxs = (gxs,)
      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          add_func(x.creator)

  def cleargrad(self):
    self.grad = None

In [23]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


In [24]:
import weakref
class Function : 
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    # 별표를 붙여 언팩
    ys = self.forward(*xs)
    # 튜플이 아닌 경우 추가 지원
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys ]
    # x 중에 가장 큰 세대 다음이니까
    self.generation = max([x.generation for x in inputs])
    for output in outputs :
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]
  
  def forward(self,xs):
    raise NotImplementedError()

  def backward(self,gys):
    raise NotImplementedError()

In [25]:
class Add(Function):
  def forward(self,x0,x1):
    y = x0 + x1 
    return y
  def backward(self, gy):
    return gy,gy

In [26]:
def add(x0, x1):
  return Add()(x0,x1)

In [27]:
class Square(Function):
  def forward(self,x):
    y = x**2
    return y
    
  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2*x*gy
    return gx

In [28]:
def square(x):
  return Square()(x)

### 17.5 동작 확인

In [29]:
for i in range(10):
  x = Variable(np.random.randn(10000))
  y = square(square(square(x)))

In [32]:
y.data

array([1.03484037e+01, 3.13421624e-03, 1.24823194e-03, ...,
       2.05239535e+01, 3.80723402e-02, 1.68253928e+01])

In [33]:
x.data

array([-1.33924234,  0.48642526, -0.43354772, ..., -1.45892386,
       -0.66462427, -1.42313353])